<a href="https://colab.research.google.com/github/JulieYego/is2/blob/main/IS_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install pytorch-lightning
!pip install Datasets

In [3]:
from datasets import Dataset
import datasets

In [4]:
import os #provides functions for interacting with the operating system
import pandas as pd
import tensorflow as tf
import nltk
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import requests
from tqdm import tqdm
from nltk.corpus import stopwords

In [5]:
from transformers import XLMRobertaTokenizer
from transformers import pipeline
model_type = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_type)

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [6]:
#TrainingArguments to specify where to save the checkpoints from training
#
from transformers import AutoModel, AutoModelForSequenceClassification, TrainingArguments, Trainer, AdamW,get_cosine_schedule_with_warmup

In [ ]:
!pip install evaluate

In [8]:
import evaluate

In [100]:
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torch.nn as nn
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F

In [10]:
from sklearn.model_selection import train_test_split 

In [11]:
#Path to the dataset
data_path = "/content/drive/MyDrive/Project/IS II Project/Datasets/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv"

In [ ]:
trans_part1 = '/content/drive/MyDrive/Colab Notebooks/Trans/Trans_part1.csv'
trans_part1_df = pd.read_csv(trans_part1)
trans_part1_df

In [ ]:
trans_p1 = '/content/drive/MyDrive/Colab Notebooks/Trans/TransP1.csv'
trans_p1_df = pd.read_csv(trans_p1)
trans_p1_df

In [ ]:
trans_p2 = '/content/drive/MyDrive/Colab Notebooks/Trans/NewTransP2.csv'
trans_p2_df = pd.read_csv(trans_p2)
trans_p2_df

In [ ]:
df1 = trans_p1_df
df2 = trans_p2_df
# append method
result = df1.append(df2)
display(result)

In [16]:
result = result.reset_index(drop=True)

In [ ]:
new_result = result[(result['comment_text']=='#ERROR!')].index
result.drop(new_result ,inplace = True)
result

In [18]:
result = result.reset_index(drop=True)

In [ ]:
result

In [ ]:
Sofia = result.iloc[10260:10270]
Sofia

In [ ]:
result['target'] = (result[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].sum(axis=1) >= 1).astype(int)
result[result['target']==1]

In [ ]:
my_test = result[['id', 'Swahili']]
my_test

In [ ]:
my_testset = result[['id', 'Swahili','toxic','severe_toxic',	'obscene',	'threat',	'insult',	'identity_hate','target']]
my_testset

In [ ]:
test_df = my_testset.rename(columns = {"Swahili": "comment_text"})
test_df

In [ ]:
test_df['labels'] = list(zip(test_df.toxic.tolist(), test_df.severe_toxic.tolist(), test_df.obscene.tolist(), test_df.threat.tolist(),  test_df.insult.tolist(), test_df.identity_hate.tolist(),test_df.target.tolist()))
test_df

In [26]:
my_test_csv = my_test.to_csv('/content/drive/MyDrive/Colab Notebooks/Trans/my_test.csv')

In [ ]:
my_test_labels = result[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate','target']]
my_test_labels

In [28]:
my_test_labels_csv = my_test_labels.to_csv('/content/drive/MyDrive/Colab Notebooks/Trans/my_test_labels.csv')

In [29]:
# default CSV
csv_data = result.to_csv('/content/drive/MyDrive/Colab Notebooks/Trans/trans_test2.csv')

In [30]:
data_df = pd.read_csv(data_path)

In [ ]:
data_df['target'] = (data_df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].sum(axis=1) >= 1).astype(int)
data_df[data_df['target']==1]

In [ ]:
Sofia_Grades = data_df.iloc[94990:95000]
Sofia_Grades

In [33]:
train_df = data_df.sample(frac=0.8, random_state=42)
val_df = data_df.drop(train_df.index)

print(f"No. of training examples: {train_df.shape[0]}")
print(f"No. of validation examples: {val_df.shape[0]}")

# No. of training examples: 120
# No. of testing examples: 30

No. of training examples: 178839
No. of validation examples: 44710


In [ ]:
train_df['labels'] = list(zip(train_df.toxic.tolist(), train_df.severe_toxic.tolist(), train_df.obscene.tolist(), train_df.threat.tolist(),  train_df.insult.tolist(), train_df.identity_hate.tolist(),train_df.target.tolist()))
train_df

In [ ]:
val_df['labels'] = list(zip(val_df.toxic.tolist(), val_df.severe_toxic.tolist(), val_df.obscene.tolist(), val_df.threat.tolist(),  val_df.insult.tolist(), val_df.identity_hate.tolist(),val_df.target.tolist()))
val_df

In [ ]:
data_df.tail()

In [ ]:
train_df.tail()

In [ ]:
val_df.head()

In [ ]:
my_test.head()

In [ ]:
data_df['comment_text'][48311]

In [ ]:
#Check the labels
train_df[train_df.columns[2:]].iloc[10000]

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
#Check for duplicates
train_df.duplicated().sum()

In [ ]:
train_df.isnull().sum()

In [ ]:
#Bar chart to visualize distribution of classes
my_attributes = train_df.columns[[2,3,4,5,6,7]]
train_df[my_attributes].sum().plot.bar()

In [ ]:
#Pie chart to visualize distribution of classes
my_labels = train_df.columns[[2,3,4,5,6,7]]
values = train_df.iloc[:,[2,3,4,5,6,7]].sum().values
mycolors = ['yellow','lightblue','orange','lightcoral','pink','seagreen']

fig = plt.figure(figsize =(10, 7))
plt.pie(values,labels = my_labels,colors = mycolors)
plt.show()

In [ ]:
#Convert to Hugging Face dataset type
train_set = Dataset.from_pandas(train_df)
train_set

In [ ]:
val_set = Dataset.from_pandas(val_df)
val_set

In [ ]:
test_set = Dataset.from_pandas(test_df)
test_set

In [ ]:
train_set = train_set.remove_columns("__index_level_0__")
train_set

In [ ]:
val_set = val_set.remove_columns("__index_level_0__")
val_set

In [53]:
my_datasets = datasets.DatasetDict({
    'train': train_set,
    'valid': val_set,
    'test' : test_set
})

In [ ]:
my_datasets

Tokenization

In [55]:
comments = train_df['comment_text']

In [ ]:
labels = train_df['labels']
labels

In [ ]:
labels_tensor = torch.FloatTensor(labels) 
labels_tensor  

In [58]:
attributes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult','identity_hate','target']

In [ ]:
labelz = train_df[attributes]
labelz

In [ ]:
val_set['labels']

In [61]:
#Function to tokenize the dataset
def tokenize(comments):
    return tokenizer(comments, #comment to encode
                     add_special_tokens = True, #add the </s> token at the beginning and the end
                     max_length = 512, #max length of the tokens
                     padding="max_length", #pad the comment to max_length
                     truncation=True, #truncate the comment to max_length
                     return_attention_mask = True, #create attention masks for [PAD] tokens
                     return_tensors = 'pt' #return pytorch tensors
                     ) 

In [62]:
def tokens(comments):
    tokenized = tokenizer.encode_plus(comments['comment_text'],
                        add_special_tokens = True, #add the </s> token at the beginning and the end
                        max_length = 512, #max length of the tokens
                        padding="max_length", #pad the comment to max_length
                        truncation=True, #truncate the comment to max_length
                        return_attention_mask = True, #create attention masks for [PAD] tokens
                        return_tensors = 'pt' #return pytorch tensors
                        ) 
    return {'input_ids': tokenized.input_ids.flatten(), 'attention_mask': tokenized.attention_mask.flatten()}
 

In [63]:
def tokens_test(comments):
    tokenized = tokenizer.encode_plus(comments['Swahili'],
                        add_special_tokens = True, #add the </s> token at the beginning and the end
                        max_length = 512, #max length of the tokens
                        padding="max_length", #pad the comment to max_length
                        truncation=True, #truncate the comment to max_length
                        return_attention_mask = True, #create attention masks for [PAD] tokens
                        return_tensors = 'pt' #return pytorch tensors
                        ) 
    return {'input_ids': tokenized.input_ids.flatten(), 'attention_mask': tokenized.attention_mask.flatten()}

In [64]:
my_tokenized_datasets = my_datasets.map(tokens)

  0%|          | 0/178839 [00:00<?, ?ex/s]

  0%|          | 0/44710 [00:00<?, ?ex/s]

  0%|          | 0/94854 [00:00<?, ?ex/s]

In [ ]:
my_tokenized_datasets.column_names

In [ ]:
my_tokenized_datasets.remove_columns(["comment_text"])

In [67]:
my_tokenized_datasets_train = my_tokenized_datasets.remove_columns(['id','comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'target'])

In [ ]:
my_tokenized_datasets_train

In [69]:
my_tokenized_datasets_train.set_format("torch")

In [ ]:
my_tokenized_datasets_train

In [71]:
small_train_dataset = my_tokenized_datasets_train["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = my_tokenized_datasets_train["test"].shuffle(seed=42).select(range(1000))

In [ ]:
small_train_dataset[0]

In [110]:
small_eval_dataset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 1000
})

In [74]:
#custom dataset class
class my_dataset(Dataset): #inherit from the Dataset class 
  def __init__(self,my_data,tokenizer,attributes): #load data
    self.my_data = my_data
    self.tokenizer = tokenizer 
    self.attributes = attributes
    
  def __len__(self):#returns length of dataset
    return len(self.my_data)

  def __getitem__(self,index):
    item = self.my_data.iloc[index]
    comment = str(item.comment_text)
    attributes = torch.FloatTensor(item[self.attributes])
    tokens = self.tokenizer.encode_plus(comment,
                                        add_special_tokens = True, #add the </s> token at the beginning and the end
                                        max_length = 512, #max length of the tokens
                                        padding = 'max_length',#pad the comment to max_length
                                        truncation = True, #truncate the comment to max_length                                    
                                        return_attention_mask = True, #create attention masks for [PAD] tokens
                                        return_tensors = 'pt', #return pytorch tensors
                                        )
    return {'input_ids': tokens.input_ids.flatten(), 
            'attention_mask': tokens.attention_mask.flatten(), 
            'labels': attributes}

In [75]:
#create an object of class my_dataset
train_ds = my_dataset(train_df, tokenizer, attributes = attributes)
val_ds = my_dataset(val_df,tokenizer,attributes = attributes)
test_ds = my_dataset(test_df,tokenizer,attributes = attributes)

In [76]:
train_ds.__getitem__(0)['labels'].shape, train_ds.__getitem__(0)['input_ids'].shape, train_ds.__getitem__(0)['attention_mask'].shape

(torch.Size([7]), torch.Size([512]), torch.Size([512]))

In [77]:
len(train_ds)

178839

In [78]:
len(val_ds)

44710

In [79]:
len(test_ds)

94854

In [ ]:
#sample of the dataset with the input id, attention mask and the label
train_ds.__getitem__(0)

In [ ]:
val_ds.__getitem__(0)

In [ ]:
test_ds.__getitem__(0)

In [109]:
#check the number of samples
total_samples = len(train_ds)
total_samples

178839

In [84]:
#create a dataloader for the dataset
train_data_loader = DataLoader(train_ds,batch_size=8, num_workers=2, shuffle=True,drop_last=True)
val_data_loader = DataLoader(val_ds,batch_size=8, num_workers=2, shuffle=True,drop_last=True)
test_data_loader = DataLoader(test_ds,batch_size=8, num_workers=2, shuffle=True,drop_last=True)

In [85]:
#create a dataloader for the dataset
train_dl = DataLoader(my_tokenized_datasets_train["train"],batch_size=8, num_workers=2, shuffle=True,drop_last=True)
val_dl = DataLoader(my_tokenized_datasets_train["valid"],batch_size=8, num_workers=2,drop_last=True)
test_dl = DataLoader(my_tokenized_datasets_train["test"],batch_size=8, num_workers=2, shuffle=True,drop_last=True)

In [113]:
#create a dataloader for the dataset
small_train_data_loader = DataLoader(small_train_dataset,batch_size=8,shuffle=True)
small_val_data_loader = DataLoader(small_eval_dataset,batch_size=8)
small_test_data_loader = DataLoader(test_ds,batch_size=8, num_workers=2, shuffle=True,drop_last=True)

In [87]:
len(train_data_loader)

22354

In [88]:
len(train_dl)

22354

In [89]:
len(val_data_loader)

5588

In [96]:
len(test_data_loader)

11856

Model

In [91]:
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=7)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [97]:
torch.cuda.empty_cache()

In [98]:
#Connect to the GPU
device = torch.device("cuda")

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [106]:
#optimizer and learning rate scheduler
from torch.optim import AdamW
optimizer = AdamW(model.parameters(),lr = 5e-6)

In [114]:
#define hyperparameters
num_epochs = 10
training_steps = num_epochs * len(small_train_data_loader)

In [115]:
training_steps

1250

In [ ]:
#Create default learning rate scheduler
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [92]:
#where to save checkpoints from training
training_args = TrainingArguments(output_dir="test_trainer")

In [93]:
metric = evaluate.load("accuracy")

In [94]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis = -1)
    return metric.compute(predictions = predictions, references = labels)

In [95]:
def build_model(transformer, max_len = 512):
    input_word_ids = Input(shape = (max_len,), dtype=tf.int32, name = "input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]    # 0 refers to output for the [CLS] token OR [all sentences,token(0 for CLS),hidden units output]
    out = Dense(1, activation = 'sigmoid')(cls_token)
    
    model = Model(inputs = input_word_ids, outputs = out)
    model.compile(Adam(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model